# Project guidé de segmentation des vaisseaux

- A faire en monome ou binôme (pas plus de 2 étudiants)
- Indiquez NOM-prénom des 1 ou 2 personnes impliquées 
- La soumission doit se faire sur EDUNAO

Pierrick Bournez Antoine Viaille
Hugues Talbot Février 2025

## Ajout: Conclusion des résultats
AJOUTER A La FIN
|       Méthode       | Partie | score Dice moyen | min | max | std |
|:-------------------|:----------------:|:----------------:|:----------------:|:----------------:|:----------------:|
|  Seuillage simple 0.05  |        2       |        0.67      |        0.54       |        0.74       |        0.06       |
|  Seuillage Otsu % image |        2       |        0.65       |        0.38       |        0.75       |        0.10       |
| $\{\text{seuillage}, \text{taille disque}\}$ optimisé |        3.1       |        0.67       |        0.58       |        0.74       |        0.05       |
| **Amélioration débruitage** | 3.2 | 
|  non local mean (nlm) |        3.2       |        0.66       |        0.47       |        0.76       |        0.08       |
|  nlm paramètres opt. |        3.2       |        0.67       |        0.48       |        0.76       |        0.08       |
|  mm denoising |        3.2       |        0.69       |        0.53      |        0.76       |        0.06       |
| **Approches segmentation** | 3.3-3.5 | 
| approche par gradient | 3.3 | 0.48 | 0.41 | 0.53 | 0.04 | 
| approche avec higra | 3.4 | 0.68 | 0.55 | 0.73 | 0.05 |  
| filtre de Frangi | 3.5 | 0.88 |0.86 | 0.89 | 0.006 |
| filtre de Sato | 3.5 | **0.98** | 0.975 | 0.98 | 0.001 |


In [ ]:
# # ## La bonne façon d'installer cv2, à ne réaliser que si besoin

# !pip3 uninstall opencv-contrib-python opencv-python
# !pip3 install opencv-contrib-python
# !pip3 install imagecodecs

In [ ]:
## Ne pas executer à l'intérieur de Visual Studio Code
# %matplotlib notebook
## A l'intérieur de Google Colab, utiliser plutôt
# %matplotlib inline

# Projet guidé, segmentation de vaisseaux sanguins

Les vaisseaux sanguins sont très difficiles à identifier et segmenter sur des images car se sont des objets "fins" : ils ont une orientation locale privilégiée, et deviennent généralement plus petit en diamètre que la résolution du capteur. Ils disparaissent donc progressivement dans le bruit de celui-ci.

L'objectif de ce projet est de realiser une segmentation d'images de vaisseaux de fond d'oeil avec des performances acceptables par des moyens classiques, y compris la morphologie mathématique.

In [ ]:
!python -m pip install -U scikit-image

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import skimage
import cv2
import imageio.v2 as iio
import imagecodecs

In [ ]:
### To help viewing images

## to view a single image
def imview(image, cmap="gray", interpolation="nearest", figsize=(6, 6)):
    plt.figure(figsize=figsize)
    plt.imshow(image, cmap=cmap, interpolation=interpolation)
    plt.axis("off")
    plt.show()


## to view several images at once
def viewlist(images, cmap="gray", figsize=(18, 6), titles=None):
    plt.figure(figsize=(len(images) * 6, 6))
    columns = len(images)
    for i, image in enumerate(images):
        plt.subplot(1, columns, i + 1)
        plt.imshow(image, cmap=cmap)
        plt.axis("off")
        if titles is not None:
            plt.title(titles[i])
    plt.show()

### Vaisseaux rétiniens

Les images de fond d'oeil sont acquises en routine chez les ophtalmologues à chaque visite. On peut y déceler de nombreuses pathologies, celles affectant la rétine mais aussi d'autres telles l'hypertension arterielle ou le diabète, même à un stade peu développé. C'est une des rares modalités où il est relativement facile de déceler les vaisseaux capillaires.

On utilise ici une petite base de donnée de 20 images. L'illustration des méthodes se fait sur l'une d'entre elles mais vous devez faire tourner vos methodes sur les 20 images. La base comprend les images de fond d'oeil, un masque de la zone d'acquisition et un masque binaire des vaisseaux segmentés à la main par un spécialiste. On cherche à s'approcher de la qualité de la segmentation manuelle.

Même dans le cas où on souhaiterait utiliser une approche par apprentissage, il est essentiel de réaliser une normalisation des images pour éviter les variations de couleur, illumination, niveau de bruit intenpestifs.

Voici une stratégie générale:

- Trouver le masque de la zone d'acquisition, car en pratique clinique, celui-ci n'est pas donné. 
- Rehausser l'apparence des vaisseaux, avec par exemple un filtre médian et/ou un top-hat noir (voir ci-dessous) 
- Réduire le bruit sans détruire les vaisseaux. Des filtres connectifs pourraient être utiles.
- Trouver un seuillage automatique
- Optimiser les paramètres de ces approches pour s'approcher le plus possible du résultat de segmentation manuel.

Essayez vos stratégiques sur 1-3 images, testez sur la base toute entière de 20 images.

In [ ]:
## Lecture d'une image
img1 = iio.imread("Eye_fundus/images/21_training.tif")
## masque d'acquisition
gtmask1 = iio.imread("Eye_fundus/mask/21_training_mask.gif")
## annotation manuelle du masque des vaisseaux. Notez qu'il n'est pas parfait mais vous allez vous rendre compte que c'est difficile de s'en approcher.
manual1 = iio.imread("Eye_fundus/1st_manual/21_manual1.gif")

In [ ]:
viewlist((img1, gtmask1, manual1))

## 1- Segmentation du masque binaire

Ça devrait être assez facile car le contraste est assez net....

In [ ]:
# Conversion en niveaux de gris
from skimage.color import rgb2gray

imgg1 = rgb2gray(img1)
print("Min = ", np.min(imgg1), "Max = ", np.max(imgg1))

imview(imgg1)


binmask1 = imgg1 > 0.1
imview(binmask1)

## Mesure de la qualité de la segmentation

La qualité d'une segmentation est souvent donnée par la pseudo-métrique de Dice, qui vaut 0 si une segmentation de référence et celle proposée n'ont rien en commun, et 1 si elles sont identiques.

Cette métrique est la même que la métrique F1 d'une matrice de confusion.

$$ 
\begin{matrix}
                & \text{True} & \text{False} \\
\text{Positive} &     TP      & FP           \\
\text{Negative} &     FN      & TN           \\
\end{matrix}
$$

Les vérité de terrain sont en colonne et les estimations en ligne

Le score F1 ou la métrique de Dice sont données par la formule
$$
Dice(A,B) = \frac{2 TP}{TP+FP+FN}
$$

On note que les vrais négatifs ne sont pas utilisés. C'est normal, car dans une segmentation le plus souvent le fond est beaucoup plus grand que la forme en surface. Utiliser les vrais négatifs dans une mesure de précision par exemple donnerait des résultats faussement optimistes.

In [ ]:
def dice(im1, im2):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0

    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(bool)
    im2 = np.asarray(im2).astype(bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2.0 * intersection.sum() / (im1.sum() + im2.sum())

In [ ]:
## Qualité de la segmentation réalisée

print("Quality of the mask segmentation:", dice(gtmask1, binmask1))

## anything over 99% is pretty much indistinguishable from the ground truth

## Question 1 : vérifier et si nécessaire améliorez la qualité de la segmentation du masque sur toute la base de donnée

Reportez le Dice moyen

In [ ]:
## Question 1: Votre code ici
dices_mask = []
for i in range(21, 41):  # the data only lies from 21 to 40
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    imgg = rgb2gray(img)
    binmask = imgg > 0.1
    dices_mask.append(dice(gtmask, binmask))

print(
    "Average Dice Mask value: ",
    np.mean(dices_mask),
    f" on the  {len(dices_mask)} images.",
)

__answer__: On a un Dice Mask moyen de 0.99 ce qui est très bien, on ne doit pas à priori améliorer la qualité de la segmentation

## 2- Preprocessing de l'image de fond d'oeil

On pourra utiliser le masque segmenter pour réduire les effets de bord.

Première idée simple: un top-hat noir. D'après le cours de morphologie mathématique, une fermerture $\varphi_B$ avec un élément structurant $B$ est extensive.
On calcule $BTH$ de la façon suivante

$$
BTH(I) = \varphi_B(I) - I
$$

Ce résidu est donc une image positive

In [ ]:
from skimage.morphology import closing, disk

imgclo1 = closing(imgg1, disk(11))

imview(imgclo1)

## black top hat

bth1 = imgclo1 - imgg1
imview(bth1)

In [ ]:
## Seuillage simpliste

vessels1 = (bth1 > 0.05) * binmask1
viewlist((binmask1, vessels1, manual1))

In [ ]:
## Mesure des performances de cette approche

print("Vessel segmentation quality", dice(manual1, vessels1))

## Question 2: automatiser et faire tourner cette approche sur la base de 20 images

Le seuillage est manuel, mais un seuillage automatique serait sans doute meilleur. 

voir par exemple [cette page](https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_thresholding.html), qui fait partie de la documentation de Scikit-image

Rapportez le Dice moyen.

In [ ]:
## Question 2: Votre code pour la segmentation des vaisseaux sur la base de données.


dices_ostu = []
dices_004 = []
for i in range(21, 41):
    # Lecture des images
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual1 = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    imgg = rgb2gray(img)

    # masque / ouverture
    binmask = imgg > 0.1
    imgclo = closing(imgg, disk(11))
    bth = imgclo - imgg

    # seuillage
    otsu_threshold = skimage.filters.threshold_otsu(bth * binmask)
    vessels1 = (bth > otsu_threshold) * binmask
    vessels2 = (bth > 0.04) * binmask

    # affichage et dice
    score_dice_otsu = dice(manual1, vessels1)
    score_dice_004 = dice(manual1, vessels2)
    dices_ostu.append(score_dice_otsu)
    dices_004.append(score_dice_004)
    print("------ Image ", i, f" (dice Otsu {score_dice_otsu}) ------")
    viewlist(
        (imgg, imgclo1, manual1, bth * binmask, vessels1, vessels2),
        titles=[
            "GT Image",
            "Ouverture",
            "GT Manuel",
            "Black top hat",
            f"Seuillage Otsu {score_dice_otsu}",
            f"Seuillage 0.04 {score_dice_004}",
        ],
    )

print(
    f" Dice Otsu moyen sur les {len(dices_ostu)} images:  {np.mean(dices_ostu)}. Le meilleur dice est de {np.max(dices_ostu)} sur l'image {np.argmax(dices_ostu) + 21}"
)
print(
    f"min: {np.min(dices_ostu)}, max: {np.max(dices_ostu)}, std: {np.std(dices_ostu)}"
)
print(
    f" Dice 0.04 moyen sur les {len(dices_004)} images:  {np.mean(dices_004)}. Le meilleur dice est de {np.max(dices_004)} sur l'image {np.argmax(dices_004) + 21}"
)
print(f"min: {np.min(dices_004)}, max: {np.max(dices_004)}, std: {np.std(dices_004)}")

 On remarque que le Seuillage d'Otsu n'est pas meilleur qu'un seuillage manuel. Il a en effet un score moyen de $0.65$ alors qu'un seuillage manuel de $0.04$ donne un score de $0.67$. Les méthodes ont la même moyenne, maximal mais l'estimateur manuel est plus consistent ( sa valeur minimal est 0.51 contre 0.37) On remarque que le seuillage d'Otsu garde du bruit autour des vaisseaux, donnant un résultat moins bon. 

In [ ]:
# Plotting the histogram of the black top hat image and Otsu threshold for image 25 and compute Dice score.
img25 = iio.imread("Eye_fundus/images/25_training.tif")
gtmask25 = iio.imread("Eye_fundus/mask/25_training_mask.gif")
manual25 = iio.imread("Eye_fundus/1st_manual/25_manual1.gif")
imgg25 = rgb2gray(img25)
binmask25 = imgg25 > 0.1
imgclo25 = closing(imgg25, disk(11))
bth25 = imgclo25 - imgg25
otsu_threshold = skimage.filters.threshold_otsu(bth25 * binmask25)
vessels25 = (bth25 > otsu_threshold) * binmask25
viewlist(
    (bth25, bth25 * binmask25, vessels25),
    titles=["Black top hat", "bth*seuil", "Seuillage Otsu"],
)
print("Dice score for image 25: ", dice(manual25, vessels25))

# plot histogram of the black top hat image and Otsu threshold for image 25
plt.figure(figsize=(6, 6))
plt.hist(bth25.flatten(), bins=256, range=(0, 1), density=True)
plt.title("Histogramme")
plt.axvline(otsu_threshold, color="r")
plt.yscale("log")
plt.show()

## 3- Optionnel: améliorer votre score

Les questions 1 et 2 sont les seules obligatoires. En option, vous pouvez tenter d'améliorer votre score moyen

## 3.1: Ajout:  Optimisation du diamètre du disque et du seuillage 
On va optimiser les deux paramètres: diamètre du disque et seuil. 

In [ ]:
training_start = 31
training_end = 41
test_start = 21
test_end = 31

In [ ]:
# plotting a heatmap of the combination of the sizes of disk and threshold values for all images
from tqdm import tqdm

dices = {}
disk_sizes = np.arange(1, 17, 1)
thresholds = np.arange(0.01, 0.25, 0.01)
for disk_size in tqdm(disk_sizes):
    for threshold in thresholds:
        dices[(disk_size, threshold)] = []
        for i in range(training_start, training_end):
            img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
            gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
            manual1 = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
            imgg = rgb2gray(img)
            binmask = imgg > 0.1
            imgclo = closing(imgg, disk(disk_size))
            bth = imgclo - imgg
            vessels = (bth > threshold) * binmask
            score_dice = dice(manual1, vessels)
            dices[(disk_size, threshold)].append(score_dice)

In [ ]:
# On a choisi un disque de taille 16
dices2 = {k: v for k, v in dices.items() if k[0] <= 16}
dices_array = np.array(
    [
        [np.mean(dices2[(disk, threshold)]) for threshold in thresholds]
        for disk in disk_sizes[:16]
    ]
)

On regarde maintenant les résultats sur le score de dice en fonction de la taille et du seuil:

In [ ]:
# dices = np.array(list(dices.values()))
dices3 = dices_array.reshape(16, 24)
plt.imshow(dices3, cmap="hot", interpolation="nearest")
plt.colorbar()
plt.xlabel("Threshold")
plt.xticks(np.arange(0, len(thresholds), 5), np.round(thresholds[::5], 2), rotation=45)
plt.ylabel("Taille disque")
plt.title("Heatmap score Dice")
plt.show()

In [ ]:
# zoom on 0.01 to 0.1 for threshold
dices_array_zoom = dices_array[:, :10]
plt.imshow(dices_array_zoom, cmap="hot", interpolation="nearest")
plt.colorbar()
plt.xlabel("Threshold")
plt.xticks(np.arange(0, 10), np.round(thresholds[:10], 2), rotation=45)
plt.ylabel("Taille disque")
plt.title("Heatmap score Dice")
plt.show()

In [ ]:
# get best disk size and threshold couple
best_disk_index, best_threshold_index = np.unravel_index(
    np.argmax(dices_array, axis=None), dices_array.shape
)
best_disk = disk_sizes[best_disk_index]
best_threshold = thresholds[best_threshold_index]
print(
    f"Meilleur couple (disk, threshold): ({best_disk}, {best_threshold}) avec un score de {dices_array[best_disk_index, best_threshold_index]}"
)

Le heatmap indique que l'argument critique est le seuillage. Nous avons arrêté le calcul du disque taille 16 car l'algorithme prenait trop de temps sans trop de différence de résultat.  Nous avons trouvé comme meilleur paramètre $\{\text{seuillage}, \text{taille disque}\}$  sur le jeu de données d'entrainements la valeur : $(6,0.04)$ 

In [ ]:
# On teste maintenant sur les images de test
dices = []
step = 3
for i in range(test_start, test_end):
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual1 = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    imgg = rgb2gray(img)
    binmask = imgg > 0.1
    imgclo = closing(imgg, disk(best_disk))
    bth = imgclo - imgg
    vessels = (bth > best_threshold) * binmask
    score_dice = dice(manual1, vessels)
    dices.append(score_dice)
    print("------ Image ", i, f" (dice {score_dice}) ------")
    if i % step == 0:
        viewlist(
            (imgg, imgclo, manual1, bth * binmask, vessels),
            titles=[
                "GT Image",
                "Ouverture",
                "GT Manuel",
                "Black top hat",
                f"Seuillage Otsu {score_dice}",
            ],
        )

print(
    f" Dice moyen sur les {len(dices)} images:  {np.mean(dices)}. Le meilleur dice est de {np.max(dices)} sur l'image {np.argmax(dices) + test_start}"
)
print(f"min: {np.min(dices)}, max: {np.max(dices)}, std: {np.std(dices)}")

__conclusion__: Le résultat moyen est maintenant 0.68 ( légèrement meilleur que 0.67 notre dernier résultat ) Mais il y a encore trop de bruit. On va essayer les autres méthodes

### 3.1- Trying some denoising

In [ ]:
from skimage.morphology import area_opening, area_closing


def mmdenoise(img, iter, clofirst=True):
    imgi = img.copy()
    for i in range(1, iter):
        if clofirst:
            imgc = area_closing(imgi, i)
            imgo = area_opening(imgc, i)
            imgi = imgo
        else:
            imgo = area_opening(imgi, i)
            imgc = area_closing(imgo, i)
            imgi = imgc
        print(".", end="")
    return imgi

In [ ]:
# On reload au cas où
img1 = iio.imread(f"Eye_fundus/images/{training_start}_training.tif")
gtmask1 = iio.imread(f"Eye_fundus/mask/{training_start}_training_mask.gif")
manual1 = iio.imread(f"Eye_fundus/1st_manual/{training_start}_manual1.gif")
imgg1 = rgb2gray(img1)
binmask1 = imgg1 > 0.1

In [ ]:
imgg1d = mmdenoise(imgg1, 8, clofirst=False)

In [ ]:
viewlist((imgg1, imgg1d))

In [ ]:
def simpleseg(fundus, mask, diameter, threshold):
    imgclo1 = closing(fundus, disk(diameter))
    bth1 = (imgclo1 - fundus) >= threshold
    return bth1 * mask

In [ ]:
ss1 = simpleseg(imgg1, binmask1, 13, 0.05)
ssd1 = simpleseg(imgg1d, binmask1, 13, 0.035)  ## with denoising, we can threshold lower
viewlist(
    (manual1, ss1, ssd1),
    titles=["GT", "Simple Segmentation", "Simple Segmentation Denoised"],
)

In [ ]:
print("Simple segmentation not denoise score =", dice(manual1, ss1))
print("Simple segmentation, denoised.  score =", dice(manual1, ssd1))

Denoising seems to help. You can try various denoising methods available in scikit image, look in particular to Non-local means

In [ ]:
from skimage.restoration import denoise_nl_means

imgg1nld = denoise_nl_means(
    imgg1, patch_size=9, h=0.01, patch_distance=22, preserve_range=True
)  ## it is worth it to explore the parameters of this method

ssnld1 = simpleseg(imgg1nld, binmask1, 13, 0.02)

viewlist((imgg1, imgg1nld, ssnld1))

print("Score of the non-local-means denoising method= ", dice(manual1, ssnld1))

Nous allons maintenant essayer d'optimiser la moyenne non locale avec le patch size et la distance de patch 

In [ ]:
# On peut aussi évaluer la méthode mmdenoise sur les images 21 à 31
dices = []
step = 3
for i in range(test_start, test_end):
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual1 = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    imgg = rgb2gray(img)
    binmask = imgg > 0.1
    imgg_denoised = mmdenoise(imgg, 8, clofirst=False)
    ss = simpleseg(imgg_denoised, binmask, 13, 0.035)
    score_dice = dice(manual1, ss)
    dices.append(score_dice)

print(
    f" Dice moyen sur les {len(dices)} images:  {np.mean(dices)}. Le meilleur dice est de {np.max(dices)} sur l'image {np.argmax(dices) + test_start}"
)
print(f"min: {np.min(dices)}, max: {np.max(dices)}, std: {np.std(dices)}")

In [ ]:
# On optimise les paramètres de la méthode de denoising et on trace une heatmap du dice score sur cette image
dices = {}
patch_sizes = np.arange(5, 15, 1)
hs = np.arange(0.001, 0.1, 0.01)
for patch_size in tqdm(patch_sizes):
    for h in hs:
        imgg1nld = denoise_nl_means(
            imgg1, patch_size=patch_size, h=h, patch_distance=22, preserve_range=True
        )
        ssnld1 = simpleseg(imgg1nld, binmask1, 13, 0.02)
        score_dice = dice(manual1, ssnld1)
        dices[(patch_size, h)] = score_dice

In [ ]:
best_patch_size, best_h = max(dices, key=dices.get)
print(
    f"Meilleur couple (patch_size, h): ({best_patch_size}, {best_h}) avec un score de {dices[(best_patch_size, best_h)]}"
)

In [ ]:
# On peut maintenant évaluer cette méthode sur les images 31 à 40
dices = []
for i in range(test_start, test_end):
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual1 = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    imgg = rgb2gray(img)
    binmask = imgg > 0.1
    imgg_nld = denoise_nl_means(
        imgg,
        patch_size=best_patch_size,
        h=best_h,
        patch_distance=22,
        preserve_range=True,
    )
    ss = simpleseg(imgg_nld, binmask, 13, 0.02)
    score_dice = dice(manual1, ss)
    dices.append(score_dice)


print(
    f" Dice moyen sur les {len(dices)} images:  {np.mean(dices)}. Le meilleur dice est de {np.max(dices)} sur l'image {np.argmax(dices) + test_start}"
)
print(f"min: {np.min(dices)}, max: {np.max(dices)}, std: {np.std(dices)}")

In [ ]:
# déterminons la patch distance
dices_patch = {}
manual1 = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
patch_distances = np.arange(15, 30, 1)
for patch_distance in tqdm(patch_distances):
    imgg1nld = denoise_nl_means(
        imgg1,
        patch_size=best_patch_size,
        h=best_h,
        patch_distance=patch_distance,
        preserve_range=True,
    )
    ssnld1 = simpleseg(imgg1nld, binmask1, 13, 0.02)
    score_dice = dice(manual1, ssnld1)
    dices_patch[patch_distance] = score_dice

best_patch_distance = max(dices_patch, key=dices_patch.get)
print(
    f"Meilleur patch_distance: {best_patch_distance} avec un score de {dices_patch[best_patch_distance]}"
)

In [ ]:
# test on images 21 to 31 avec tous les meilleurs paramètres
dices_nld = []
for i in range(test_start, test_end):
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    imgg = rgb2gray(img)
    binmask = imgg > 0.1
    imgg_nld = denoise_nl_means(
        imgg,
        patch_size=best_patch_size,
        h=best_h,
        patch_distance=best_patch_distance,
        preserve_range=True,
    )
    ss = simpleseg(imgg_nld, binmask, 13, 0.02)
    score_dice = dice(manual, ss)
    dices_nld.append(score_dice)

print(
    f" Dice moyen sur les {len(dices_nld)} images:  {np.mean(dices_nld)}. Le meilleur dice est de {np.max(dices_nld)} sur l'image {np.argmax(dices_nld) + 31}"
)
print(f"min: {np.min(dices_nld)}, max: {np.max(dices_nld)}, std: {np.std(dices_nld)}")

 Nous avons donc optimisé les paramètres de la méthode moyenne non locale ce qui donne un meilleur coupe (patch_size,h) de (5,0.001) basé sur l'image 31 .
 Testé sur la base de test, on obtient un Dice de 0.37.  On optimise alors la patch distance pour trové une distance de 0.37 qui es très semblable au résultat initial. ¨Pour les images en test avec l'algorithme de denoising, ce choix manuel semblent fortement détérioré la qualité de ségmentation 
 Avec mmdenoise on a un score de 0.67 ce qui est semblable au dénoising manuel.
 Mais on  n'a fait que faire du preprocessing à l'algorithme de segmentation, on pourrait voir le résultat si on modifie le résultat de l'algorithme

## 3.2- Using a learned gradient

In [ ]:
!pip install higra

In [ ]:
import higra as hg
import cv2


import urllib.request as request

exec(
    request.urlopen(
        "https://github.com/higra/Higra-Notebooks/raw/master/utils.py"
    ).read(),
    globals(),
)


detector = cv2.ximgproc.createStructuredEdgeDetection(get_sed_model_file())

img1f = img1.astype(np.float32) / 255  ## for this we use the colour image as input

gradient_image = detector.detectEdges(img1f)

imview(gradient_image)

In [ ]:
## we need to use a shrunk mask

from skimage.morphology import erosion, opening

smask1 = erosion(binmask1, disk(15))

gradvessel1 = gradient_image * smask1
imview(gradvessel1)

In [ ]:
## use a white top-hat on the gradient

gradvesseld1 = gradvessel1 - opening(gradvessel1, disk(2)) > 0.01
imview(gradvesseld1)

In [ ]:
print("Score of the learned gradient method= ", dice(manual1, gradvesseld1))

On optimise maintenant le disque d'érosion et d'ouverture.

In [ ]:
# make a grid of plt.subplots with size of disk of erosion and opening pamaraters
# for every combination of erosion and opening, take the best threshold and then compute the dice score
# show threshold and dice on title of subfigure

dices = {}
images_save = {}  # used for subplot grid
disk_sizes_erosion = np.arange(1, 11, 1)
disk_sizes_opening = np.arange(1, 11, 1)
thresholds = np.arange(0.01, 0.1, 0.01)
gradient_image = detector.detectEdges(img1f)

for disk_size_e in disk_sizes_erosion:
    for disk_size_o in tqdm(disk_sizes_opening, leave=False):
        smask1 = erosion(binmask1, disk(disk_size_e))
        gradvessel = gradient_image * smask1
        current_best_dice = 0
        best_threshold = 0
        for threshold in thresholds:
            gradvesseld = (
                gradvessel - opening(gradvessel, disk(disk_size_o)) > threshold
            )
            score_dice = dice(manual1, gradvesseld)
            if score_dice > current_best_dice:
                current_best_dice = score_dice
                best_threshold = threshold

        dices[(disk_size_e, disk_size_o)] = current_best_dice
        # recompute gradvesseld with best threshold for image
        gradvessel = gradient_image * smask1
        gradvesseld = (
            gradvessel - opening(gradvessel, disk(disk_size_o)) > best_threshold
        )
        images_save[(disk_size_e, disk_size_o)] = gradvesseld

best_disk_size_e, best_disk_size_o = max(dices, key=dices.get)
print(
    f"Meilleur couple (disk_size_e, disk_size_o): ({best_disk_size_e}, {best_disk_size_o}) avec un score de {dices[(best_disk_size_e, best_disk_size_o)]}"
)

In [ ]:
# make the subplot grid
fig, axs = plt.subplots(
    len(disk_sizes_erosion), len(disk_sizes_opening), figsize=(20, 20)
)
for i, disk_size_e in enumerate(disk_sizes_erosion):
    for j, disk_size_o in enumerate(disk_sizes_opening):
        axs[i, j].imshow(images_save[(disk_size_e, disk_size_o)])
        axs[i, j].set_title(
            f"e: {disk_size_e}, o: {disk_size_o}, Dice: {round(dices[(disk_size_e, disk_size_o)], 2)}"
        )
        axs[i, j].axis("off")

In [ ]:
dices = []
for i in range(test_start, test_end):
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    imgf = img.astype(np.float32) / 255
    gradient_image = detector.detectEdges(imgf)
    smask = erosion(binmask, disk(best_disk_size_e))
    gradvessel = gradient_image * smask

    current_best_dice = 0
    best_threshold = 0
    for threshold in thresholds:
        gradvesseld = (
            gradvessel - opening(gradvessel, disk(best_disk_size_o)) > threshold
        )
        score_dice = dice(manual, gradvesseld)
        if score_dice > current_best_dice:
            current_best_dice = score_dice
            best_threshold = threshold

    gradvessel = gradient_image * smask
    gradvesseld = (
        gradvessel - opening(gradvessel, disk(best_disk_size_o)) > best_threshold
    )
    dices.append(dice(manual, gradvesseld))

print(
    f" Dice moyen sur les {len(dices)} images:  {np.mean(dices)}. Le meilleur dice est de {np.max(dices)} sur l'image {np.argmax(dices) + test_start}"
)
print(f"min: {np.min(dices)}, max: {np.max(dices)}, std: {np.std(dices)}")

__conclusion__: On a le plus petit écart type pour le moment mais un Dice décevant: cette technique n'apporte rien de plus. Un des problèmes est ici que nous obtenons comme positif le contour de l'oeil (et donc FP ce qui fait baisser le Dice). Visuellement, il y a beaucoup de veines qui ne sont pas segmentées donc même en supprimant cet effet on n'aura pas un bon score. Concernant l'effet des deux paramètres, nous obtenons les effets attendus concernant la taille du disque de l'érosion et de la dilatation.
Nous allons donc passer à une approche par connectivité pour tenter d'améliorer le Dice. 

## 3.3- using a connective approach

Here we use the interactive approach from the higra tutorial. Try to find a good tree and a good rule, then apply it on the whole database

In [ ]:
## start from the black top-hat image

imgclo1 = closing(imgg1, disk(15))
bth1 = (imgclo1 - imgg1) * smask1  ## to avoid border effects

size = bth1.shape
print("Image size:", size)
graph = hg.get_4_adjacency_graph(size)
tree, altitudes = hg.component_tree_max_tree(graph, bth1)

In [ ]:
# from ipywidgets import interact, FloatSlider


def subtractive_rule(tree, altitudes, deleted_nodes):
    # altitudes difference with parent node
    delta_altitudes = altitudes - altitudes[tree.parents()]
    # remove deleted nodes
    delta_altitudes[deleted_nodes] = 0
    # restore root absolute alitudes
    delta_altitudes[tree.root()] = altitudes[tree.root()]
    # accumulated deltas without deleted nodes
    altitudes = hg.propagate_sequential_and_accumulate(
        tree, delta_altitudes, hg.Accumulators.sum
    )

    return altitudes, deleted_nodes


trees = {
    "Min Tree": hg.component_tree_min_tree(graph, bth1),
    "Max Tree": hg.component_tree_max_tree(graph, bth1),
    "Tree of Shapes": hg.component_tree_tree_of_shapes_image2d(bth1),
}

attributes = {
    "area": lambda tree, altitudes: hg.attribute_area(tree),
    "height": lambda tree, altitudes: hg.attribute_height(tree, altitudes) * 4,
    "compactness": lambda tree, altitudes: hg.attribute_compactness(tree) * 1000,
}

## Compute relevant features
filtering_rule = {
    "direct": lambda tree, altitudes, deleted_nodes: (altitudes, deleted_nodes),
    "min": lambda tree, altitudes, deleted_nodes: (
        altitudes,
        hg.propagate_sequential(tree, deleted_nodes, np.logical_not(deleted_nodes)),
    ),
    "max": lambda tree, altitudes, deleted_nodes: (
        altitudes,
        hg.accumulate_and_min_sequential(
            tree, deleted_nodes, np.ones((tree.num_leaves(),)), hg.Accumulators.min
        ),
    ),
    "subtractive": subtractive_rule,
}


## Taken from tutorial
def filtering(sTree, sAttr, sRule, sThreshold):
    tree, altitudes = trees[sTree]
    attribute = attributes[sAttr](tree, altitudes)

    deleted_nodes = attribute < sThreshold

    altitudes, deleted_nodes = filtering_rule[sRule](tree, altitudes, deleted_nodes)

    result = hg.reconstruct_leaf_data(tree, altitudes, deleted_nodes)

    imshow(result, cmap="gray")


# interact(filtering,
#         sTree=['Min Tree', 'Max Tree', 'Tree of Shapes'],
#         sAttr=['area', 'height', 'compactness'],
#         sRule=['direct', 'min', 'max', 'subtractive'],
#         sThreshold=FloatSlider(min=0, max=1000, step=1, continuous_update=False));

In [ ]:
score_dice

In [ ]:
# make a combination of subtractive rule, attributes and trees to find the best dice score
# make a grid search of the best parameters
# show the heatmap of the dice score for every sRule in filtering_rule. Every time find best threshold


for sRule in filtering_rule.keys():
    dices = {}
    for sTree in trees.keys():
        for sAttr in attributes.keys():
            best_threshold = 0
            current_best_dice = 0
            for sThreshold in tqdm(np.arange(0, 1000, 10), leave=False):
                tree, altitudes = trees[sTree]
                attribute = attributes[sAttr](tree, altitudes)
                deleted_nodes = attribute < sThreshold
                altitudes, deleted_nodes = filtering_rule[sRule](
                    tree, altitudes, deleted_nodes
                )
                result = hg.reconstruct_leaf_data(tree, altitudes, deleted_nodes)
                bin_result = result > 0.05
                score_dice = dice(manual1, bin_result)
                if score_dice > current_best_dice:
                    current_best_dice = score_dice
                    best_threshold = sThreshold
            dices[(sTree, sAttr)] = current_best_dice

    dices_array = np.array(list(dices.values()))
    dices_array = dices_array.reshape(len(trees), len(attributes))
    plt.imshow(dices_array, cmap="hot", interpolation="nearest")
    plt.colorbar()
    plt.xlabel("Attributes")
    plt.xticks(np.arange(0, len(attributes), 1), list(attributes.keys()), rotation=45)
    plt.ylabel("Trees")
    plt.yticks(np.arange(0, len(trees), 1), list(trees.keys()), rotation=45)
    plt.title(f"Dice pour {sRule}")
    plt.show()

In [ ]:
imgclo1 = closing(imgg1, disk(15))
bth1 = (imgclo1 - imgg1) * smask1  ## to avoid border effects

size = bth1.shape
print("Image size:", size)
graph = hg.get_4_adjacency_graph(size)
tree, altitudes = hg.component_tree_max_tree(graph, bth1)
size = bth1.shape
print("Image size:", size)
graph = hg.get_4_adjacency_graph(size)
tree, altitudes = hg.component_tree_max_tree(graph, bth1)

In [ ]:
dices = []
sTree = trees["Max Tree"]
sAttr = attributes["area"]
sRule = filtering_rule["subtractive"]

for i in range(test_start, test_end):
    # ouverture classique
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")

    #  preprocessing
    binmask = imgg > 0.1
    imgg = rgb2gray(img)
    imgclo = closing(imgg, disk(15))
    smask = erosion(binmask, disk(best_disk_size_e))
    bth = (imgclo - imgg) * smask

    # hg
    size = bth1.shape
    graph = hg.get_4_adjacency_graph(size)
    tree, altitudes = hg.component_tree_max_tree(graph, bth)
    sAttr = hg.attribute_area(tree)
    sTree = tree
    sRule = subtractive_rule

    # boucle pour trouver le meilleur threshold
    best_threshold = 0
    current_best_dice = 0
    for sThreshold in np.arange(0, 1000, 10):
        attribute = sAttr
        deleted_nodes = attribute < sThreshold
        altitudes, deleted_nodes = sRule(tree, altitudes, deleted_nodes)
        result = hg.reconstruct_leaf_data(tree, altitudes, deleted_nodes)
        bin_result = result > 0.04
        score_dice = dice(manual, bin_result)
        if score_dice > current_best_dice:
            current_best_dice = score_dice
            best_threshold = sThreshold
    dices.append(current_best_dice)

print(
    f" Dice moyen sur les {len(dices)} images:  {np.mean(dices)}. Le meilleur dice est de {np.max(dices)} sur l'image {np.argmax(dices) + test_start}"
)
print(f"min: {np.min(dices)}, max: {np.max(dices)}, std: {np.std(dices)}")

On a fait ici toutes les combinaisons possibles entre les arbres et attributs, et on a repris le seuil à $0.04$ pour tester binariser les vaisseaux comme c'était le seuil qui marchait le mieux auparavant. Le meilleur Dice est obtenu pour la combinaison max tree et area. 
On remarque que le Dice obtenu est semblable avec avec le seuillage manuel. Cela ne marche pas dans notre cas. Nous allons maintenant essayer avec les filtres plus classiques et adaptés à notre problème: filtres de Frangi et Sato. 

## 3.5 Frangi et Sato

In [ ]:
# Imports nécessaires

from skimage.filters import frangi
from skimage.morphology import disk
from skimage.filters import rank
from skimage.morphology import square
from skimage.segmentation import watershed

In [ ]:
# load image 31, apply frangi filter and show the result

img21 = iio.imread(f"Eye_fundus/images/{training_start}_training.tif")
gtmask21 = iio.imread(f"Eye_fundus/mask/{training_start}_training_mask.gif")
manua21 = iio.imread(f"Eye_fundus/1st_manual/{training_start}_manual1.gif")
img21 = rgb2gray(img21)
binmask21 = img21 > 0.1
img21_frangi = frangi(img21 * binmask21)
viewlist((img21, img21_frangi), titles=["Original", "Frangi"])

Le filtre de Frangi donne des résultats étranges, on va preprocess l'image

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
img21_clahe = clahe.apply((img21 * 255).astype(np.uint8))
frangi_clahe = frangi(img21_clahe)
frangi_clahe_mask = frangi_clahe * binmask21
dice_frangi = dice(gtmask21, frangi_clahe)
dice_frangi_mask = dice(gtmask21, frangi_clahe_mask)
viewlist(
    (img21, img21_clahe, frangi_clahe, frangi_clahe_mask),
    titles=[
        "Originale",
        "CLAHE",
        f"Frangi (dice: {dice_frangi})",
        f"Frangi mask (dice: {dice_frangi_mask})",
    ],
)

In [ ]:
dices = []
for i in range(test_start, test_end):
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    img = rgb2gray(img)
    binmask = img > 0.1
    img_clahe = clahe.apply((img * 255).astype(np.uint8))
    frangi_clahe = frangi(img_clahe)
    frangi_clahe_mask = frangi_clahe * binmask
    score_dice = dice(gtmask, frangi_clahe_mask)
    dices.append(score_dice)

print(
    f" Dice moyen sur les {len(dices)} images:  {np.mean(dices)}. Le meilleur dice est de {np.max(dices)} sur l'image {np.argmax(dices) + test_start}"
)
print(f"min: {np.min(dices)}, max: {np.max(dices)}, std: {np.std(dices)}")

In [ ]:
# make Sato filtre on image 21
from skimage.filters import sato


img21 = iio.imread(f"Eye_fundus/images/{training_start}_training.tif")
gtmask21 = iio.imread(f"Eye_fundus/mask/{training_start}_training_mask.gif")
manual21 = iio.imread(f"Eye_fundus/1st_manual/{training_start}_manual1.gif")
img21_gray = rgb2gray(img21)  # Convert image to grayscale
binmask21 = img21_gray > 0.1
img21_clahe = clahe.apply((img21_gray * 255).astype(np.uint8))
sato_clahe = sato(img21_clahe)
sato_clahe_mask = sato_clahe * binmask21
dice_sato = dice(gtmask21, sato_clahe)
dice_sato_mask = dice(gtmask21, sato_clahe_mask)
viewlist(
    (img21, manual21, img21_clahe, sato_clahe, sato_clahe_mask),
    titles=[
        "Originale",
        "GT",
        "CLAHE",
        f"Sato (dice: {dice_sato})",
        f"Sato mask (dice: {dice_sato_mask})",
    ],
)

In [ ]:
dices = []
step = 3
for i in range(test_start, test_end):
    img = iio.imread(f"Eye_fundus/images/{i}_training.tif")
    gtmask = iio.imread(f"Eye_fundus/mask/{i}_training_mask.gif")
    manual = iio.imread(f"Eye_fundus/1st_manual/{i}_manual1.gif")
    img = rgb2gray(img)
    binmask = img > 0.1
    img_clahe = clahe.apply((img * 255).astype(np.uint8))
    sato_clahe = sato(img_clahe)
    sato_clahe_mask = sato_clahe * binmask
    score_dice = dice(gtmask, sato_clahe_mask)
    dices.append(score_dice)
    # print and view list of images
    print(f"------ Image {i} (dice: {score_dice}) ------")
    if i % step == 0:
        viewlist(
            (img, manual, img_clahe, sato_clahe, sato_clahe_mask),
            titles=["Originale", "GT", "CLAHE", f"Sato", f"Sato mask"],
        )

print(
    f" Dice moyen sur les {len(dices)} images:  {np.mean(dices)}. Le meilleur dice est de {np.max(dices)} sur l'image {np.argmax(dices) + test_start}"
)
print(f"min: {np.min(dices)}, max: {np.max(dices)}, std: {np.std(dices)}")

Nous voyons bien ici que la méthode de Frangi est très adaptée à noter problème car il y a une grande amélioration dans le Dice ( on est maintenant à $0.88$). Elle est quand même nettement en dessous de la méthode de Sato qui donne un Dice de $0.98$ sur la première image. EN testant sur les images de test on a un résultat $0.97$.

### 3.4 Approche par apprentissage

La base de données est plutôt petite, donc un apprentissage classique par U-Net par exemple peut ne pas fonctionner très bien. D'autre part les annotations de la partie fine des vaisseaux sanguins (vers les extrémités) est quasiment négligeable par rapport au reste. 

Pour améliorer les résultats, on peut utiliser une approche par apprentissage avec une partie morphologique, en suivant l'article 

https://arxiv.org/pdf/2404.03010

Il est possible **(optionnellement !)** d'augmenter et vérifier son travail sur une BDD différente de celle de DRIVE, voir par exemple

https://pmc.ncbi.nlm.nih.gov/articles/PMC10219065/pdf/jcm-12-03587.pdf


### Critique du Dice

La métrique Dice est largement utilisée en imagerie médicale, mais elle présente certaines limitations. En particulier, il est crucial de supprimer correctement le contour de l'œil, sous peine d'obtenir un score Dice biaisé en raison de la grande taille de cette région. Certaines méthodes tendent à supprimer davantage les vaisseaux ainsi que cette zone, ce qui réduit artificiellement le nombre de faux positifs et peut induire en erreur lors du choix d'une méthode, comme observé dans la dernière cellule.

De plus, cette métrique repose sur une vérité terrain, qui varie naturellement en fonction du médecin qui effectue l'annotation. Une approche plus avancée et réaliste consisterait à intégrer l'expertise de plusieurs médecins pour générer une carte de densité de vérité terrain. Cette dernière pourrait alors servir de référence, à l’instar des jeux de données de segmentation, afin d’obtenir une évaluation plus robuste des résultats.


__conclusion__:

La conclusion et notre cheinement se trouve en début de notebook. Voici les techniques que nous avons mises en place: 

Vous pouvez tenter tout ou partie des approches suivantes:

- ✔️Try to optimize the parameters of the approach, maybe on half the database, test on the other half
    - The diameter of the disk
    - The threshold
    - Try to improve the mask, it may help

- ✔️Try some connective approach, under the assuption that the vessel network is connected, using **higra**
- ✔️Try using a well-researched gradient instead or before the top hat, eg the Dollar gradient (see `cv2.ximgproc.createStructuredEdgeDetection` from `higra_tutoria_01`)
- ✔️Try using some denoising and image simplification
- ✔️Try using the *Vesselness* filters from scikit-image, [especially designed to enhance vessels](https://scikit-image.org/docs/stable/api/skimage.filters.html). 
    - Try the Frangi filter and 
    - Try the Sato tubeness enhancement.
    They both require some parameter tuning
- ✔️Take a critical approach to the metric. Is Dice the best metric? Are the annotation of good enough quality ?
